In [1]:
## misc
import os
import sys
import math
import time
import h5py
import pickle
import numpy as np
import pandas as pd
import random 
import sqlite3
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.family'] = 'IPAPGothic'
%matplotlib inline

import keras
from keras.models import Model, Sequential, load_model
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.pooling import GlobalAveragePooling2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard, CSVLogger
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras import backend as K
from keras.utils import to_categorical
from PIL import Image

import sklearn
from sklearn.metrics import confusion_matrix, classification_report, log_loss
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.datasets import load_iris
from sklearn import datasets, linear_model
from sklearn.externals import joblib
from IPython.display import display
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta


from tqdm import tqdm_notebook as tqdm
from tqdm import tnrange
tqdm().pandas()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>")) 

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

def p(a):
    display(a)
def print_confusion_matrix(y_valid, y_pred):
    labels = sorted(list(set(y_valid)))
    cmx_data = confusion_matrix(y_valid, y_pred, labels=labels)
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
    plt.figure()
    sns.heatmap(df_cmx, annot=True)
    plt.show()
def print_evaluation(y_valid, y_pred):
    print(metrics.classification_report(y_valid, y_pred))
    print("")
    print("accuracy        : %f" % metrics.accuracy_score(y_valid, y_pred))
    print("")
    print("precision(binary): %f" % metrics.precision_score(y_valid, y_pred, average='binary'))
    print("recall(binary)   : %f" % metrics.recall_score(y_valid, y_pred, average='binary'))
    print("f1(binary)       : %f" % metrics.f1_score(y_valid, y_pred, average='binary'))
    print("")
    print("precision(macro): %f" % metrics.precision_score(y_valid, y_pred, average='macro'))
    print("recall(macro)   : %f" % metrics.recall_score(y_valid, y_pred, average='macro'))
    print("f1(macro)       : %f" % metrics.f1_score(y_valid, y_pred, average='macro'))
    print("")
def print_auc(fpr, tpr, _):
    print("auc             : %f" % metrics.auc(fpr, tpr))
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %.2f)' % metrics.auc(fpr, tpr))
    plt.legend()
    plt.title('ROC curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    return metrics.auc(fpr, tpr)
def print_feature_importance(columns, model):
    cf = pd.DataFrame(index=columns)
    cf["feature_importances"] = model.feature_importances_.round(6)
    if len(columns) == len(model.feature_importances_):
        cf = cf.sort_values('feature_importances', ascending=False)
        plt.figure(figsize=(10, 70))
        sns.barplot(x='feature_importances', y=cf.index, data=cf)
        return model.feature_importances_
def print_coef(columns, model):
    cf = pd.DataFrame(index=columns)
    cf['coef'] = model.coef_[0].round(6)
    if len(columns) == len(model.coef_[0]):
        cf = cf.sort_values('coef', ascending=False)
        plt.figure(figsize=(10, 70))
        sns.barplot(x='coef', y=cf.index, data=cf)
        return cf['coef']

np.random.seed(0)
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
PREFIX = 'tmp/06.'

sys.path.append(os.getcwd() + '/HirosakiAnalyze')
import iwakic as iwk
import iwakic_outcome as iwko
import iwakic_item as iwki
# import iwakic_datarobot as iwkdr

Using TensorFlow backend.


In [2]:
# 項目名リストを読み込む
SourceAllItemListFileName = '/mnt/u-kyoto/HirosakiData/iwaki_mv181003_ALLItemsList.csv'
ItemList = iwk.read_and_prepare_ItemList(SourceAllItemListFileName)
ItemList.head()

reading item list file...
Done!


,item_lv1,item_lv2,item_lv3,item_eng,item_type,item_unit,item_attr,min_value,max_value
item_name_other,,,,,,,,,
Ⅳ型コラーゲン,NaN,NaN,NaN,NaN,continuous,NaN,NaN,100,99
1000kHz-LA Impedance,NaN,NaN,NaN,NaN,continuous,NaN,NaN,154.1,459
1000kHz-LL Impedance,NaN,NaN,NaN,NaN,continuous,NaN,NaN,131.4,368
1000kHz-RA Impedance,NaN,NaN,NaN,NaN,continuous,NaN,NaN,153.7,490.2
1000kHz-RL Impedance,NaN,NaN,NaN,NaN,continuous,NaN,NaN,127.4,318.5


In [3]:
# 元データを3次元でピボットしたdata arrayを読み込む
Source_pickle_FileName = '/mnt/u-kyoto/HirosakiData/da_181003_rm_str.pickle'

if os.path.isfile(Source_pickle_FileName):
    da = iwk.read_dataarray_pickle(Source_pickle_FileName) #pickle fileが存在すれば読み込む。なければ作成。
else:
    TargetYear = range(2005, 2018)
    SourceFileName_in_each_TargetYear = [('/mnt/u-kyoto/HirosakiData/iwaki_mv181003_' + str(x) + '.csv') for x in TargetYear]
    sr = iwk.make_alldata_series(TargetYear, SourceFileName_in_each_TargetYear, ItemList)
    
    bln_make_pickle = True
    Output_pickle_FileName = Source_pickle_FileName
    da = iwk.make_alldata_dataarray(sr, bln_make_pickle, Output_pickle_FileName)
da

reading DataArray pickle file...
Done!


<xarray.DataArray (year: 13, cypher_id: 3136, item_name_other: 3112)>
array([[[  nan,   nan, ...,   nan,   nan],
        [  nan,   nan, ...,   nan,   nan],
        ...,
        [  nan,   nan, ...,   nan,   nan],
        [  nan,   nan, ...,   nan,   nan]],

       [[  nan,   nan, ...,   nan,   nan],
        [  nan,   nan, ...,   nan,   nan],
        ...,
        [  nan,   nan, ...,   nan,   nan],
        [  nan,   nan, ...,   nan,   nan]],

       ...,

       [[  nan,   nan, ...,   nan,   nan],
        [  nan,   nan, ...,   nan,   nan],
        ...,
        [  nan,   nan, ...,   nan,   nan],
        [  nan,   nan, ...,   nan,   nan]],

       [[  nan,   nan, ...,   nan,   nan],
        [310.6, 209.6, ...,   0. ,   nan],
        ...,
        [  nan,   nan, ...,   nan,   nan],
        [212.8, 190.8, ...,   0. ,   nan]]])
Coordinates:
  * year             (year) int64 2005 2006 2007 2008 ... 2014 2015 2016 2017
  * cypher_id        (cypher_id) object '01zVVyBw' '02gISino' ... 'zziRvgoA'
 

In [4]:
# 上記daに項目情報を加えたDatasetを作成
ds = iwk.make_alldata_Dataset(da, ItemList)
ds

making xr.Dataset...
Done!


<xarray.Dataset>
Dimensions:          (cypher_id: 3136, item_name_other: 3112, year: 13)
Coordinates:
  * year             (year) int64 2005 2006 2007 2008 ... 2014 2015 2016 2017
  * cypher_id        (cypher_id) object '01zVVyBw' '02gISino' ... 'zziRvgoA'
  * item_name_other  (item_name_other) object '1000kHz-LA Impedance' ... 'ﾀｹﾌﾟﾛﾝ、ﾊﾟﾘｴｯﾄ2.その他3.わからない'
Data variables:
    vals             (year, cypher_id, item_name_other) float64 nan nan ... nan
    item_type        (item_name_other) object 'continuous' ... 'nominal'
    item_lv1         (item_name_other) object nan nan nan nan ... nan nan nan
    item_lv2         (item_name_other) object nan nan nan nan ... nan nan nan
    item_lv3         (item_name_other) object nan nan nan nan ... nan nan nan
    item_eng         (item_name_other) object nan nan nan nan ... nan nan nan
    item_unit        (item_name_other) object nan nan nan nan ... nan nan nan
    item_attr        (item_name_other) object nan nan nan nan ... nan nan nan

In [5]:
# 計算等で付加する項目を追加
item_eGFR = iwki.Item_eGFR()
ds = item_eGFR.add_calculated_value_to_ds(ds)
ds

<xarray.Dataset>
Dimensions:          (cypher_id: 3136, item_name_other: 3113, year: 13)
Coordinates:
  * year             (year) int64 2005 2006 2007 2008 ... 2014 2015 2016 2017
  * cypher_id        (cypher_id) object '01zVVyBw' '02gISino' ... 'zziRvgoA'
  * item_name_other  (item_name_other) object '1000kHz-LA Impedance' ... 'eGFR'
Data variables:
    vals             (year, cypher_id, item_name_other) float64 nan ... 88.41
    item_type        (item_name_other) object 'continuous' ... 'continuous'
    item_lv1         (item_name_other) object nan nan nan nan ... nan nan nan
    item_lv2         (item_name_other) object nan nan nan nan ... nan nan nan
    item_lv3         (item_name_other) object nan nan nan nan ... nan nan nan
    item_eng         (item_name_other) object nan nan nan nan ... nan nan nan
    item_unit        (item_name_other) object nan nan nan nan ... nan nan nan
    item_attr        (item_name_other) object nan nan nan nan ... nan nan nan

In [6]:
# 前年との差分を追加
ds = iwk.add_delta_to_ds(ds, ItemList)
ds

calculating delta values...
merging to Dataset...
Done!


<xarray.Dataset>
Dimensions:          (cypher_id: 3136, item_name_other: 5936, year: 13)
Coordinates:
  * item_name_other  (item_name_other) object '1000kHz-LA Impedance' ... 'ﾀｹﾌﾟﾛﾝ、ﾊﾟﾘｴｯﾄ2.その他3.わからない'
  * year             (year) int64 2005 2006 2007 2008 ... 2014 2015 2016 2017
  * cypher_id        (cypher_id) object '01zVVyBw' '02gISino' ... 'zziRvgoA'
Data variables:
    vals             (year, cypher_id, item_name_other) float64 nan nan ... nan
    item_type        (item_name_other) object 'continuous' ... 'nominal'
    item_lv1         (item_name_other) object nan nan nan nan ... nan nan nan
    item_lv2         (item_name_other) object nan nan nan nan ... nan nan nan
    item_lv3         (item_name_other) object nan nan nan nan ... nan nan nan
    item_eng         (item_name_other) object nan nan nan nan ... nan nan nan
    item_unit        (item_name_other) object nan nan nan nan ... nan nan nan
    item_attr        (item_name_other) object nan nan nan nan ... nan nan nan

In [7]:
class DM(iwk.Disease):
    def __init__(self, ds):
        super().__init__(ds)

    dis_name = 'DM'
    rm_items = ['健康状況_医師_糖尿病', '健康状況_服薬_糖尿病治療薬', '糖尿病', '糖尿病_産婦人科', '糖尿病治療薬',
                '血清血糖', 'HbA1c_NGSP', 'HbA1c_JDS', 'HbA1c', '問7_HbA1c', 'グリコアルブミン',
                '糖尿病_治療開始年齢', '既往歴糖尿病']
    rm_items.extend(['delta_' + x for x in rm_items])

    def diag_rule(self, ar, item_names):
        self.logic_1 = self.targetval(ar, item_names, '健康状況_医師_糖尿病') == 1 #以下は1ならば(+)、それ以外はあってもかわらない
        self.logic_2 = self.targetval(ar, item_names, '健康状況_服薬_糖尿病治療薬') == 1
        self.logic_3 = self.targetval(ar, item_names, '糖尿病') == 1
        self.logic_3_2 = self.targetval(ar, item_names, '既往歴糖尿病') == 1 #"治療中"
        self.logic_4 = self.targetval(ar, item_names, '糖尿病_産婦人科') == 1
        self.logic_5 = self.targetval(ar, item_names, '糖尿病治療薬') == 1

        self.logic_1to5 = np.array([self.logic_1, self.logic_2, self.logic_3, self.logic_3_2, self.logic_4, self.logic_5])

        self.logic_7 = self.targetval(ar, item_names, '血清血糖') >= 126
        self.logic_8 = self.targetval(ar, item_names, 'HbA1c_NGSP') >= 6.5
        self.logic_9 = self.targetval(ar, item_names, 'HbA1c_JDS') >= 6.1
        self.logic_10 = self.targetval(ar, item_names, 'HbA1c') >= 6.1

        self.logic_7_na = np.isnan(self.targetval(ar, item_names, '血清血糖'))
        self.logic_8_na = np.isnan(self.targetval(ar, item_names, 'HbA1c_NGSP'))
        self.logic_9_na = np.isnan(self.targetval(ar, item_names, 'HbA1c_JDS'))
        self.logic_10_na = np.isnan(self.targetval(ar, item_names, 'HbA1c'))


        if np.any(self.logic_1to5): #logic_1~5で1があれば(+)で確定
            self.res = 1.0
        elif np.any([self.logic_7, self.logic_8, self.logic_9, self.logic_10]):
            self.res = 1.0
        elif self.logic_7_na | all([self.logic_8_na, self.logic_9_na, self.logic_10_na]): #(logic_7がnan)または(8,9,10のすべてがnan)ならnan
            self.res = np.nan
        else:
            self.res = 0.0

        return(self.res)

In [8]:
dm = DM(ds)
dm.da_diag.to_pandas().transpose()

year,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
cypher_id,,,,,,,,,,,,,
01zVVyBw,0.0,0.0,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
02gISino,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
08uSGGlI,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0
09vHG4b8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
0AD8QmH8,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0EOpKVOB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
0GOrYl02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0GrYBzst,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN
0H9egO2W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN


In [9]:
dm.print_timeseries_diag_pattern()

All Neg:2769
Pos Exist:358
New Onset Exist:134


In [10]:
# df_PTのitem_type:nominalの項目において、値->カテゴリ名に置き換える
def Replace_NominalValue_to_CategoryName(df_PT, NominalValueDictSourceFileName, ds, dis, drop_rm_items):
    NominalValueDict = pd.read_csv(NominalValueDictSourceFileName)

    #valueに整数値以外が混ざっていたので削除（BaseBraの"L"）
    Bln_value_is_digit = [x.isdigit() if isinstance(x, str) else True for x in NominalValueDict['value'].tolist()]
    NominalValueDict = NominalValueDict.iloc[Bln_value_is_digit,:]

    NominalValueDict['value'] = NominalValueDict['value'].astype(np.float).astype(np.str)

    tmp = ds.item_type.to_pandas()
    nominal_item_name_other = tmp[tmp=='nominal'].index


    if drop_rm_items:
        target_nominal_item_name_other = nominal_item_name_other[np.logical_not(nominal_item_name_other.isin(dis.rm_items))]
    else:
        target_nominal_item_name_other = nominal_item_name_other

    def Convert_value_to_category(sr): #ひとつのSeries(カラム)を変換する関数
        Cur_dict = NominalValueDict.query('item_name_other == @sr.name')
        sr = sr.astype(np.str)
        sr.replace(Cur_dict['value'].tolist(), Cur_dict['category_name'].tolist(), inplace=True)
        return(sr)

    df_PT_nominal = df_PT.loc[:,target_nominal_item_name_other].apply(Convert_value_to_category, axis=0)
    df_PT_other = df_PT.iloc[:,np.logical_not(df_PT.columns.isin(target_nominal_item_name_other))]
    tmp_df = pd.concat([df_PT_other, df_PT_nominal], axis=1)
    df_PT = tmp_df[df_PT.columns] #並べ直し

    return(df_PT)

def make_dataset_and_write_csv_for_model(df_PT, ds, dis, modeltype, drop_rm_items=True, target_span=3):
    def extract_target_rec(CurRow, ds):
        res = ds.vals.loc[CurRow.year, CurRow.cypher_id, :].values
        return(res)

    print('making dataset... ')
    tmp = df_PT.apply(extract_target_rec, axis=1, args=(ds,))
    mat = np.array(tmp.tolist())
    df_PT_val = pd.DataFrame(mat, columns=ds.vals.coords['item_name_other'].values, index=df_PT.index)

    if drop_rm_items:
        df_PT_val.drop(dis.rm_items, axis=1, inplace=True, errors='ignore') # leakage防止用の除外項目リスト(dis.rm_item)の項目を抜く

    df_PT = pd.concat([df_PT, df_PT_val], axis=1)

    df_PT['year'] = pd.to_datetime(df_PT['year'], format='%Y') # yearをdatetime64型に変換（擬似的に1/1:0:00になる）

    print('converting nominal values to category names... ')
    NominalValueDictSourceFileName = '/mnt/u-kyoto/HirosakiData/iwaki_mv181003_value_dict_nominal.csv'
    df_PT = Replace_NominalValue_to_CategoryName(df_PT, NominalValueDictSourceFileName, ds, dis, drop_rm_items)

    if(modeltype=='Pos_vs_Neg_allpoints'):
        OutputFileName = 'DatasetForModel_mv181003_' + dis.dis_name + '_Pos_vs_Neg_allpoints.csv'
        dr_project_name = 'mv181003_' + dis.dis_name + '_Pos_vs_Neg_allpoints'
        dr_target_name = 'diag_' + dis.dis_name
    elif(modeltype=='New_Onset_in_x_yr_or_Not'):
        OutputFileName = 'DatasetForModel_mv181003_' + dis.dis_name + '_New_Onset_in_' + str(target_span) +'yr_or_Not.csv'
        dr_project_name = 'mv181003_' + dis.dis_name + '_New_Onset_in_' + str(target_span) +'yr_or_Not'
        dr_target_name = 'onset_in_' + str(target_span) + 'yr_' + dis.dis_name
    else:
        OutputFileName = 'DatasetForModel_mv181003_' + dis.dis_name + '_' + modeltype + '.csv'
        dr_project_name = 'mv181003_' + dis.dis_name + '_' + + modeltype
        dr_target_name = 'hoge' ##ここは未実装

    print('writing in "' + OutputFileName, '"')
    df_PT.to_csv(OutputFileName, index=False)
    print('Done!')

    dr_file_path = os.getcwd() + '/' + OutputFileName

    param_for_dr = {'project_name': dr_project_name, 'file_path': dr_file_path, 'target_name': dr_target_name}

    return([df_PT, param_for_dr])

In [11]:
modeltype='Pos_vs_Neg_allpoints'
drop_rm_items = True

df_PT = iwk.make_target_cypher_id_and_year_list_df(dm, modeltype)
df_PT, param_for_dr = make_dataset_and_write_csv_for_model(df_PT, ds, dm, modeltype, drop_rm_items)
df_PT.head()

Pos label: 1089
Neg label: 11708
making dataset... 
converting nominal values to category names... 
writing in "DatasetForModel_mv181003_DM_Pos_vs_Neg_allpoints.csv "
Done!


,year,cypher_id,diag_DM,1000kHz-LA Impedance,1000kHz-LL Impedance,1000kHz-RA Impedance,1000kHz-RL Impedance,1000kHz-TR Impedance,10m最大歩行速度,10m最大歩行速度_1回目,...,Ｆリーチ_2回,Ｆリーチ_3回,Ｆリーチ_代表値,Ｈピロリ抗体判定,Ｈピロリ抗体判定_補助コメント,Ｈピロリ抗体濃度_補助コメント,ＨＰ抗体／ＡＢＣ_判定,ＨＰ抗体／ＡＢＣ_濃度,Ｍ２ＢＰＧｉ_判定,ﾀｹﾌﾟﾛﾝ、ﾊﾟﾘｴｯﾄ2.その他3.わからない
53,2005-01-01,17Afbh8E,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,nan,nan
124,2005-01-01,2oCe7U5f,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,nan,nan
128,2005-01-01,2qeXpjso,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,nan,nan
175,2005-01-01,3jLE7iLx,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,nan,nan
177,2005-01-01,3l2MvLB8,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,nan,nan


In [12]:
if 1:
    df_PT.to_hdf(PREFIX+'tmp', 'df_PT')
df = pd.read_hdf(PREFIX+'tmp', 'df_PT')
df.head()

/home/hiroyam/.local/share/virtualenvs/data-u-kyoto-feature-_MSQN99e/lib/python3.6/site-packages/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['cypher_id', 'ACTH_補助コメント', 'Angiotensinogen_Met235Thr', 'BNP_補助コメント', 'BaseBra', 'CPITN1', 'CPITN2', 'CPITN3', 'CPITN4', 'CPITN5', 'CPITN6', 'CPITN代表値', 'CPI代表', 'CPI代表_中上', 'CPI代表_中下', 'CPI代表_右上', 'CPI代表_右下', 'CPI代表_左上', 'CPI代表_左下', 'Dis', 'FABP2_Ala54Thr', 'IBStype', 'KIR_Glu23Lys', 'Lympho_補助コメント', 'MCHC_補助コメント', 'MCH_補助コメント', 'MCV_補助コメント', 'Neutro_補助コメント', 'PPARγ_Pro12Ala', 'RAGE_G1704T', 'SUR1', 'Seg_補助コメント', 'Stab_補助コメント', 'UCP1_A-3826G', 'V', 'atrophy', 'dQ2', 'lymphotoxin_alpha_A252G', 'rs429358', 'rs7412', 'total_P1MP_分布_補助コメント', 'total_P1MP_補助コメント', 'ucOC_補助コメント', 'β2-AdrenergicReceptor', 'β3-AdrenergicReceptor', 'すい臓の病気', 'その他の病気1', 'その他の病気2', 'その他の病気3', 'その他の病気4', 'その他の病気5', '

,year,cypher_id,diag_DM,1000kHz-LA Impedance,1000kHz-LL Impedance,1000kHz-RA Impedance,1000kHz-RL Impedance,1000kHz-TR Impedance,10m最大歩行速度,10m最大歩行速度_1回目,...,Ｆリーチ_2回,Ｆリーチ_3回,Ｆリーチ_代表値,Ｈピロリ抗体判定,Ｈピロリ抗体判定_補助コメント,Ｈピロリ抗体濃度_補助コメント,ＨＰ抗体／ＡＢＣ_判定,ＨＰ抗体／ＡＢＣ_濃度,Ｍ２ＢＰＧｉ_判定,ﾀｹﾌﾟﾛﾝ、ﾊﾟﾘｴｯﾄ2.その他3.わからない
53,2005-01-01,17Afbh8E,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,nan,nan
124,2005-01-01,2oCe7U5f,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,nan,nan
128,2005-01-01,2qeXpjso,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,nan,nan
175,2005-01-01,3jLE7iLx,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,nan,nan
177,2005-01-01,3l2MvLB8,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,nan,nan


In [13]:
# 数値型カラムを平均値で埋める
mask = ['year', 'cypher_id']
numerical_columns   = (df.select_dtypes(include=[np.number]).columns | mask) ^ mask

df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].mean())
df[numerical_columns].head()

,1000kHz-LA Impedance,1000kHz-LL Impedance,1000kHz-RA Impedance,1000kHz-RL Impedance,1000kHz-TR Impedance,10m最大歩行速度,10m最大歩行速度_1回目,10m最大歩行速度_2回目,10m最大歩行速度_3回目,10m歩行採用値,...,％予測値_V50,％予測値_V75,Ｃ．Ｏ．Ｉ．,Ｆリーチ_1回,Ｆリーチ_2回,Ｆリーチ_3回,Ｆリーチ_代表値,Ｈピロリ抗体判定,ＨＰ抗体／ＡＢＣ_判定,ＨＰ抗体／ＡＢＣ_濃度
53,287.381835,218.354021,281.305393,216.9228,16.926585,4.572307,4.206414,4.150884,3.176667,3.748087,...,84.215115,99.99983,0.576364,30.76882,31.34938,31.812582,31.979559,0.291747,0.356075,7.73271
124,287.381835,218.354021,281.305393,216.9228,16.926585,4.572307,4.206414,4.150884,3.176667,3.748087,...,84.215115,99.99983,0.576364,30.76882,31.34938,31.812582,31.979559,0.291747,0.356075,7.73271
128,287.381835,218.354021,281.305393,216.9228,16.926585,4.572307,4.206414,4.150884,3.176667,3.748087,...,84.215115,99.99983,0.576364,30.76882,31.34938,31.812582,31.979559,0.291747,0.356075,7.73271
175,287.381835,218.354021,281.305393,216.9228,16.926585,4.572307,4.206414,4.150884,3.176667,3.748087,...,84.215115,99.99983,0.576364,30.76882,31.34938,31.812582,31.979559,0.291747,0.356075,7.73271
177,287.381835,218.354021,281.305393,216.9228,16.926585,4.572307,4.206414,4.150884,3.176667,3.748087,...,84.215115,99.99983,0.576364,30.76882,31.34938,31.812582,31.979559,0.291747,0.356075,7.73271


In [14]:
# カテゴリ型カラムをダミー化
categorical_columns = (df.select_dtypes(include=[np.object, np.bool]).columns | mask) ^ mask

df = pd.concat([
    df[mask],
    df[numerical_columns],
    pd.get_dummies(df[categorical_columns])
], axis=1)
df.head()

,year,cypher_id,1000kHz-LA Impedance,1000kHz-LL Impedance,1000kHz-RA Impedance,1000kHz-RL Impedance,1000kHz-TR Impedance,10m最大歩行速度,10m最大歩行速度_1回目,10m最大歩行速度_2回目,...,高血圧_罹患歴あり,Ｈピロリ抗体判定_補助コメント_nan,Ｈピロリ抗体判定_補助コメント_測定不能,Ｈピロリ抗体濃度_補助コメント_nan,Ｈピロリ抗体濃度_補助コメント_測定不能,Ｍ２ＢＰＧｉ_判定_nan,ﾀｹﾌﾟﾛﾝ、ﾊﾟﾘｴｯﾄ2.その他3.わからない_nan,ﾀｹﾌﾟﾛﾝ、ﾊﾟﾘｴｯﾄ2.その他3.わからない_その他,ﾀｹﾌﾟﾛﾝ、ﾊﾟﾘｴｯﾄ2.その他3.わからない_わからない,ﾀｹﾌﾟﾛﾝ、ﾊﾟﾘｴｯﾄ2.その他3.わからない_ｵﾒﾌﾟﾗｰﾙ、ﾀｹﾌﾟﾛﾝ、ﾊﾟﾘｴｯﾄ
53,2005-01-01,17Afbh8E,287.381835,218.354021,281.305393,216.9228,16.926585,4.572307,4.206414,4.150884,...,0,1,0,1,0,1,1,0,0,0
124,2005-01-01,2oCe7U5f,287.381835,218.354021,281.305393,216.9228,16.926585,4.572307,4.206414,4.150884,...,0,1,0,1,0,1,1,0,0,0
128,2005-01-01,2qeXpjso,287.381835,218.354021,281.305393,216.9228,16.926585,4.572307,4.206414,4.150884,...,0,1,0,1,0,1,1,0,0,0
175,2005-01-01,3jLE7iLx,287.381835,218.354021,281.305393,216.9228,16.926585,4.572307,4.206414,4.150884,...,0,1,0,1,0,1,1,0,0,0
177,2005-01-01,3l2MvLB8,287.381835,218.354021,281.305393,216.9228,16.926585,4.572307,4.206414,4.150884,...,0,1,0,1,0,1,1,0,0,0


In [15]:
# Staticカラムを削除
df = df[df.columns[df.nunique() > 1]]
df.head()

,year,cypher_id,1000kHz-LA Impedance,1000kHz-LL Impedance,1000kHz-RA Impedance,1000kHz-RL Impedance,1000kHz-TR Impedance,10m最大歩行速度,10m最大歩行速度_1回目,10m最大歩行速度_2回目,...,高血圧_罹患歴あり,Ｈピロリ抗体判定_補助コメント_nan,Ｈピロリ抗体判定_補助コメント_測定不能,Ｈピロリ抗体濃度_補助コメント_nan,Ｈピロリ抗体濃度_補助コメント_測定不能,Ｍ２ＢＰＧｉ_判定_nan,ﾀｹﾌﾟﾛﾝ、ﾊﾟﾘｴｯﾄ2.その他3.わからない_nan,ﾀｹﾌﾟﾛﾝ、ﾊﾟﾘｴｯﾄ2.その他3.わからない_その他,ﾀｹﾌﾟﾛﾝ、ﾊﾟﾘｴｯﾄ2.その他3.わからない_わからない,ﾀｹﾌﾟﾛﾝ、ﾊﾟﾘｴｯﾄ2.その他3.わからない_ｵﾒﾌﾟﾗｰﾙ、ﾀｹﾌﾟﾛﾝ、ﾊﾟﾘｴｯﾄ
53,2005-01-01,17Afbh8E,287.381835,218.354021,281.305393,216.9228,16.926585,4.572307,4.206414,4.150884,...,0,1,0,1,0,1,1,0,0,0
124,2005-01-01,2oCe7U5f,287.381835,218.354021,281.305393,216.9228,16.926585,4.572307,4.206414,4.150884,...,0,1,0,1,0,1,1,0,0,0
128,2005-01-01,2qeXpjso,287.381835,218.354021,281.305393,216.9228,16.926585,4.572307,4.206414,4.150884,...,0,1,0,1,0,1,1,0,0,0
175,2005-01-01,3jLE7iLx,287.381835,218.354021,281.305393,216.9228,16.926585,4.572307,4.206414,4.150884,...,0,1,0,1,0,1,1,0,0,0
177,2005-01-01,3l2MvLB8,287.381835,218.354021,281.305393,216.9228,16.926585,4.572307,4.206414,4.150884,...,0,1,0,1,0,1,1,0,0,0


In [16]:
df.to_hdf(PREFIX+'tmp', 'df')

In [17]:
df['diag_DM'].value_counts()

0.0    11708
1.0     1089
Name: diag_DM, dtype: int64